In [1]:
import sys
from pathlib import Path

#Path.cwd().parent depends on where you run this code from
#Path(__file__).resolve().parents[1] depends on where this file is located on disk
#
#project_root = .../1099-reconciliation-pipeline
project_root = Path.cwd().parent  # running from notebooks/  folder (cwd = current working directory) - .parent gets us to the folder above
sys.path.append(str(project_root)) #sys.path is a list of folders where python looks for modules
                                    # we add the project root to that list with .append

print(project_root)

/Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline


In [2]:
# Cell 2 — Imports, Load & clean inputs (real paths)

from src import load_data
from src.clean_matrix import clean_matrix
from src.clean_relius_roth_basis import clean_relius_roth_basis
from src.build_correction_file import build_correction_dataframe, write_correction_file

from src.age_taxcode_analysis import clean_relius_demo

from src.config import RAW_DATA_DIR

matrix_path = RAW_DATA_DIR / "real_all_matrix_2025.xlsx"
relius_roth_basis_path = RAW_DATA_DIR / "real_roth_basis_relius_2025.xlsx"
relius_demo_path = RAW_DATA_DIR / "real_demo_relius_2025.xlsx"


# Load  and clean Matrix raw data
matrix_raw = load_data.load_matrix_excel(path=matrix_path, use_sample_if_none=False)
matrix_clean = clean_matrix(matrix_raw)

# Load  and clean Relius Roth Basis raw data
relius_roth_basis_raw = load_data.load_relius_roth_basis_excel(path=relius_roth_basis_path, use_sample_if_none=False, sheet_name="Sheet5")
relius_roth_basis_clean = clean_relius_roth_basis(relius_roth_basis_raw)

# Load and clean Relius Demo raw data
relius_demo_clean = clean_relius_demo(relius_demo_path)   # currently one function to load and clean Relius raw Excel file


#.shape is an attribute of pandas DataFrames that returns a tuple of
# (number of rows, number of columns) - e.g. (1000, 15)
print("matrix_raw DataFrame:    ", matrix_raw.shape)
print("matrix_clean DataFrame:  ", matrix_clean.shape)
print("\n")
print("relius_roth_basis_raw:   ", relius_roth_basis_raw.shape)
print("relius_roth_basis_clean: ", relius_roth_basis_clean.shape)
print("\n")
print("relius_demo_clean:       ", relius_demo_clean.shape)

/Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline/src/clean_matrix.py:181: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(series, errors="coerce").dt.date


matrix_raw DataFrame:     (60972, 56)
matrix_clean DataFrame:   (6980, 16)


relius_roth_basis_raw:    (7904, 6)
relius_roth_basis_clean:  (7904, 6)


relius_demo_clean:        (61584, 6)


In [3]:
# Cell 3 — Quick view of Relius Roth Basis clean data

print("relius_roth_basis_clean Head:\n")
relius_roth_basis_clean.head(10)


relius_roth_basis_clean Head:



,plan_id,ssn,first_name,last_name,first_roth_tax_year,roth_basis_amt
0,100MBDII-R,201746826,Alyssa,Mihalik,2020,2500.0
5277,406MBDII-R,208726408,Josh,Myers,2021,9600.0
5275,406MBDII-R,167788536,Nina,Zanias-Anderson,2022,8400.0
5274,406MBDII-R,192721010,Juliana,Rappo,2018,7700.0
5273,406MBDII-R,165788891,Jacob,Augustus,2022,7700.0
5272,406MBDII-R,205769452,Zackary,Hockman,2022,7300.0
5271,406MBDII-R,530980202,Tristan,Boyd,2022,6975.0
5270,406MBDII-R,158784228,Elizabeth,Burks,2021,6650.0
5269,406MBDII-R,099763952,Katherine,Golebiewski,2023,6400.0
5268,406MBDII-R,197749337,Dana,Salanik,2023,6300.0


In [4]:
# Cell 4 — Check key data inside Relius Roth Basis clean data

print("'ssn' lenght and index:\n", relius_roth_basis_clean["ssn"].str.len().value_counts(dropna=False))
print("\n'ssn' duplicates?:\n", relius_roth_basis_clean.duplicated(["plan_id", "ssn"]).sum())

'ssn' lenght and index:
 ssn
9    7904
Name: count, dtype: int64

'ssn' duplicates?:
 0


In [5]:
relius_roth_basis_clean.dtypes

plan_id                 object
ssn                     object
first_name              object
last_name               object
first_roth_tax_year      Int64
roth_basis_amt         float64
dtype: object

In [6]:
# Cell 5 — Validate required columns exist (pre-flight)

required_matrix_cols = {
    "plan_id","ssn","txn_date","transaction_id","participant_name","matrix_account",
    "gross_amt","fed_taxable_amt","roth_initial_contribution_year","tax_code_1","tax_code_2"
}
missing = required_matrix_cols - set(matrix_clean.columns)
assert not missing, f"Matrix missing columns: {missing}"

required_demo_cols = {"plan_id","ssn","dob"}
missing = required_demo_cols - set(relius_demo_clean.columns)
assert not missing, f"Demo missing columns: {missing}"

required_basis_cols = {"plan_id","ssn","first_roth_tax_year","roth_basis_amt"}
missing = required_basis_cols - set(relius_roth_basis_clean.columns)
assert not missing, f"Roth basis missing columns: {missing}"

print("✓ Required columns present")

✓ Required columns present


In [ ]:
# Cell 6 — Run Roth Basis Taxable Analysis Engine

from src.roth_taxable_analysis import run_roth_taxable_analysis


relius_roth_basis = run_roth_taxable_analysis(
    matrix_clean,
    relius_demo_clean,
    relius_roth_basis_clean
)

print("relius_roth_basis_df:", relius_roth_basis.shape)
relius_roth_basis.head(10)

relius_roth_basis_df: (287, 21)


,transaction_id,txn_date,ssn,participant_name,matrix_account,plan_id,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,...,gross_amt,roth_initial_contribution_year,first_roth_tax_year,roth_basis_amt,age_at_txn,suggested_taxable_amt,suggested_first_roth_tax_year,correction_reason,action,match_status
0,44745362,2025-11-20,165723517,Rose E. Frankil,07P6LM4G,300005MBD,B,1,<NA>,<NA>,...,1795.39,2008,2008,1000.0,35.635866,<NA>,<NA>,taxable_within_15pct_of_gross,INVESTIGATE,match_needs_review
1,44568463,2025-11-14,177749131,Selena Merrick,07P6LM4G,300005MBD,H,<NA>,<NA>,<NA>,...,22601.99,2015,2015,15925.0,32.856947,<NA>,<NA>,<NA>,<NA>,match_no_action
2,44568464,2025-11-14,170608226,Andrea Platts,07P6LM4G,300005MBD,H,<NA>,<NA>,<NA>,...,42774.70,2011,2011,0.0,61.158111,0.0,<NA>,qualified_roth_distribution,<NA>,match_no_action
3,43961174,2025-10-27,163581720,Jeffrey Smith,07P6LM4G,300005MBD,B,7,<NA>,<NA>,...,1111.11,2025,2018,4350.0,60.621492,0.0,2018,roth_initial_year_mismatch,UPDATE_1099,match_needs_correction
4,43886241,2025-10-23,208725197,Casey Goebel,07P6LM4G,300005MBD,H,<NA>,<NA>,<NA>,...,25257.87,2025,2017,5500.0,33.886379,<NA>,2017,roth_initial_year_mismatch,UPDATE_1099,match_needs_correction
5,43144300,2025-10-01,172647883,Kathleen Stettner,07P6LM4G,300005MBD,H,<NA>,<NA>,<NA>,...,53925.27,2009,2009,0.0,59.000684,<NA>,<NA>,<NA>,<NA>,match_no_action
6,42490716,2025-09-08,167401805,Marianne Smith,07P6LM4G,300005MBD,B,7,<NA>,<NA>,...,3000.00,2025,2016,10600.0,69.180014,0.0,2016,roth_initial_year_mismatch,UPDATE_1099,match_needs_correction
7,42302462,2025-09-02,167401805,Marianne Smith,07P6LM4G,300005MBD,B,7,<NA>,<NA>,...,3000.00,2025,2016,10600.0,69.163587,0.0,2016,roth_initial_year_mismatch,UPDATE_1099,match_needs_correction
8,42034715,2025-08-22,184581413,Kimberly Criss,07P6LM4G,300005MBD,B,4,<NA>,<NA>,...,7160.56,2017,<NA>,NaN,NaN,<NA>,<NA>,missing_first_roth_tax_year,INVESTIGATE,match_needs_review
9,41909277,2025-08-18,192448025,Mary Powers,07P6LM4G,300005MBD,H,<NA>,<NA>,<NA>,...,8189.90,2009,2009,6000.0,72.689938,0.0,<NA>,qualified_roth_distribution,<NA>,match_no_action


In [8]:
# Cell 7 — Output schema check (builder-compatible canonical fields)

required_out_cols = {
    "transaction_id","txn_date","ssn","participant_name","matrix_account",
    "tax_code_1","tax_code_2","suggested_tax_code_1","suggested_tax_code_2",
    "correction_reason","action","match_status",
    "suggested_taxable_amt","suggested_first_roth_tax_year"
}
missing = required_out_cols - set(relius_roth_basis.columns)
assert not missing, f"Engine C output missing columns: {missing}"

print("✓ Engine C output schema OK (builder-compatible)")

✓ Engine C output schema OK (builder-compatible)


In [9]:
# Cell 8 — Filter validation (Roth-only + inherited excluded)

# Roth plan check based on plan_id rules:
is_roth = relius_roth_basis["plan_id"].astype(str).str.startswith("300005") | relius_roth_basis["plan_id"].astype(str).str.endswith("R")
assert is_roth.all(), "Found non-Roth plan_id rows in Engine C output."

print("✓ Roth-only filter passed")

✓ Roth-only filter passed


In [10]:
# Cell 9 — Join coverage diagnostics (DOB + basis availability)

import pandas as pd

print("DOB missing in Engine C output:", relius_roth_basis.get("dob", pd.Series(dtype=object)).isna().sum() if "dob" in relius_roth_basis.columns else "DOB not retained")
print("first_roth_tax_year missing:", relius_roth_basis["suggested_first_roth_tax_year"].isna().sum(), "(note: this can be NA if not needed)")

DOB missing in Engine C output: DOB not retained
first_roth_tax_year missing: 170 (note: this can be NA if not needed)


In [ ]:
# Cell 10 — Validate “basis coverage” rule is actually triggering

zero_taxable = relius_roth_basis[relius_roth_basis["suggested_taxable_amt"].fillna(pd.NA).eq(0.0)]
print("Rows suggesting taxable=0:", len(zero_taxable))
zero_taxable[["plan_id","ssn","age_at_txn","gross_amt","fed_taxable_amt","roth_initial_contribution_year","first_roth_tax_year", "roth_basis_amt","suggested_first_roth_tax_year","suggested_taxable_amt","correction_reason","match_status","action"]].head(25)

Rows suggesting taxable=0: 137


,plan_id,ssn,age_at_txn,gross_amt,fed_taxable_amt,roth_initial_contribution_year,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action
2,300005MBD,170608226,61.158111,42774.70,0.00,2011,2011,0.00,<NA>,0.0,qualified_roth_distribution,match_no_action,<NA>
3,300005MBD,163581720,60.621492,1111.11,1057.36,2025,2018,4350.00,2018,0.0,roth_initial_year_mismatch,match_needs_correction,UPDATE_1099
6,300005MBD,167401805,69.180014,3000.00,2910.58,2025,2016,10600.00,2016,0.0,roth_initial_year_mismatch,match_needs_correction,UPDATE_1099
7,300005MBD,167401805,69.163587,3000.00,2910.58,2025,2016,10600.00,2016,0.0,roth_initial_year_mismatch,match_needs_correction,UPDATE_1099
9,300005MBD,192448025,72.689938,8189.90,0.00,2009,2009,6000.00,<NA>,0.0,qualified_roth_distribution,match_no_action,<NA>
11,300005MBD,202420774,74.160164,8441.36,0.00,2009,2009,4000.00,<NA>,0.0,qualified_roth_distribution,match_no_action,<NA>
12,300005MBD,196683352,44.594114,5000.00,4963.24,2025,2018,8900.00,2018,0.0,roth_initial_year_mismatch,match_needs_correction,UPDATE_1099
13,300005MBD,199383874,73.412731,12457.63,0.00,2010,<NA>,NaN,<NA>,0.0,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
14,300005MBD,175407849,74.976044,12464.49,0.00,2010,<NA>,NaN,<NA>,0.0,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
15,300005MBD,177461069,71.107461,8421.79,0.00,2012,<NA>,NaN,<NA>,0.0,missing_first_roth_tax_year,match_needs_review,INVESTIGATE


Notes:
- This checks that suggested_taxable_amt == 0 is being produced and why.
- FIX: 
    - if 'roth_basis_amt' > 'gross_amount' AND 'first_roth_tax_year' == 'roth_initial_contribution_year' -> 'no correction needed' or 'qualified_roth_distribution'
    - if ppt is older than 59 1/2 AND 'current year' >= 'roth_initial_contribution_year + '5 years' -> 'no correction needed' or 'qualified_roth_distribution'
    - if 'first_roth_tax_year' != 'roth_initial_contribution_year' -> needs_correction

In [12]:
# Cell 11 — Validate the 15% proximity flag (INVESTIGATE behavior)

investigate = relius_roth_basis[relius_roth_basis["action"].eq("INVESTIGATE")]
print("INVESTIGATE rows:", len(investigate))
investigate[["plan_id","ssn","gross_amt","fed_taxable_amt","correction_reason","match_status","action"]].head(25)

INVESTIGATE rows: 71


,plan_id,ssn,gross_amt,fed_taxable_amt,correction_reason,match_status,action
0,300005MBD,165723517,1795.39,1795.39,taxable_within_15pct_of_gross,match_needs_review,INVESTIGATE
8,300005MBD,184581413,7160.56,0.00,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
13,300005MBD,199383874,12457.63,0.00,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
14,300005MBD,175407849,12464.49,0.00,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
15,300005MBD,177461069,8421.79,0.00,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
16,300005MBD,162684618,2541.15,0.00,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
20,300005MBDII,177569732,34277.43,0.00,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
22,300005MBDII,201647061,5000.00,5000.00,taxable_within_15pct_of_gross,match_needs_review,INVESTIGATE
24,300005MBDII,984438850,1666.67,0.00,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
25,300005MBDII,856244680,39340.24,0.00,missing_first_roth_tax_year,match_needs_review,INVESTIGATE
